In [13]:
%pip install fredapi

In [2]:
"""
=============================================================
  CORN COMMODITY PRICE PREDICTION — ML PIPELINE
=============================================================
Author: Claude (Anthropic)
Description:
    End-to-end machine learning pipeline to predict monthly
    corn futures prices (CBOT front-month, $/bushel).

Data Sources (live API calls are stubbed; swap in your keys):
    - USDA NASS Quick Stats API  → acreage, yield, stocks
    - USDA WASDE/ERS             → supply/demand balances
    - NOAA CDO API               → temperature, precipitation
    - EIA API                    → ethanol production, crude oil
    - FRED API (St. Louis Fed)   → DXY, interest rates, CPI
    - CFTC COT reports           → managed money positioning
    - Quandl/Nasdaq Data Link    → corn futures prices

Model:
    Gradient Boosting Regressor (sklearn) with walk-forward
    cross-validation. Predicts next-month corn price.

=============================================================
"""

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import warnings
warnings.filterwarnings("ignore")
import requests

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline


# ─────────────────────────────────────────────────────────────
# 1.  LIVE DATA FETCHING  (commented stubs — add your API keys)
# ─────────────────────────────────────────────────────────────

def fetch_usda_nass(api_key, commodity="CORN", year_start=2000, year_end=2025):
    """
    Fetch acreage, yield, and production from USDA NASS Quick Stats.

    API docs: https://quickstats.nass.usda.gov/api
    Get a free key at: https://quickstats.nass.usda.gov/api#param_define
    """
    url = "https://quickstats.nass.usda.gov/api/api_GET/"
    params = {
        "key": api_key,
        "commodity_desc": commodity,
        "statisticcat_desc": "YIELD",
        "agg_level_desc": "NATIONAL",
        "year__GE": year_start,
        "format": "JSON"
        }
    r = requests.get(url, params=params)
    data = r.json()["data"]
    return pd.DataFrame(data)[["year","Value"]].rename(columns={"Value":"yield_bu_acre"})


def fetch_eia(api_key, series_id="PET.RWTC.M"):
    """
    Fetch EIA time series (crude oil, ethanol production, etc.)

    API docs: https://api.eia.gov/bulk/
    Series IDs:
        PET.RWTC.M       → WTI Crude Oil monthly ($/bbl)
        STEO.CORNPROD_US.M → Ethanol production proxy
        EIA-AEO.REF2023.COGEN_GEN_NA_GEN_NA_NA_ELC_BLNKWH.A → Ethanol plant capacity
    """

    url = f"https://api.eia.gov/v2/seriesid/{series_id}"
    r = requests.get(url, params={"api_key": api_key, "frequency": "monthly"})
    df = pd.DataFrame(r.json()["response"]["data"])
    df["date"] = pd.to_datetime(df["period"])
    return df.set_index("date")[["value"]].rename(columns={"value": series_id})


def fetch_fred(api_key, series_ids=("DTWEXBGS", "FEDFUNDS", "CPIAUCSL")):
    """
    Fetch macroeconomic series from FRED.

    API docs: https://fred.stlouisfed.org/docs/api/fred/
    Series IDs:
        DTWEXBGS  → Nominal Broad Dollar Index (DXY-equivalent)
        FEDFUNDS  → Federal Funds Rate
        CPIAUCSL  → Consumer Price Index
    """

    from fredapi import Fred
    fred = Fred(api_key=api_key)
    frames = {sid: fred.get_series(sid, observation_start="2000-01-01")
                  for sid in series_ids}
    return pd.DataFrame(frames).resample("MS").last()


def fetch_cftc_cot():
    """
    Download CFTC Commitments of Traders data for CORN futures.

    CFTC publishes bulk CSV files at:
    https://www.cftc.gov/MarketReports/CommitmentsofTraders/HistoricalCompressed/index.htm

    Example:
    """
    url = "https://www.cftc.gov/files/dea/history/fut_disagg_txt_hist_2006_2024.zip"
    df = pd.read_csv(url, compression="zip")
    corn = df[df["Market_and_Exchange_Names"].str.contains("CORN", na=False)]
    corn["date"] = pd.to_datetime(corn["As_of_Date_In_Form_YYMMDD"], format="%y%m%d")
    return corn.set_index("date")[["M_Money_Positions_Long_All", "M_Money_Positions_Short_All"]]


def fetch_noaa_climate(token, station_id="GHCND:USW00094846", start="2000-01-01"):
    """
    Fetch monthly temperature & precipitation from NOAA CDO API.

    API docs: https://www.ncdc.noaa.gov/cdo-web/webservices/v2
    Get a free token at: https://www.ncdc.noaa.gov/cdo-web/token
    """
    headers = {"token": token}
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    params = {
        "datasetid": "GSOM",
        "stationid": station_id,
        "startdate": start,
        "enddate": "2025-12-31",
        "datatypeid": "TAVG,PRCP",
        "limit": 1000
        }
    r = requests.get(url, headers=headers, params=params)
    df = pd.DataFrame(r.json()["results"])
    df["date"] = pd.to_datetime(df["date"])

    return df.pivot(index="date", columns="datatype", values="value")



In [25]:
# ─────────────────────────────────────────────────────────────
# 2.  SYNTHETIC DATA GENERATION
#     Mirrors real statistical properties of each source.
#     Replace each block with a live fetch call above.
# ─────────────────────────────────────────────────────────────

def generate_synthetic_dataset(start="2000-01-01", end="2025-12-31", seed=42):
    """
    Generate a realistic monthly dataset of corn price drivers.
    Statistical properties calibrated to historical USDA/NOAA/EIA data.
    """
    rng = np.random.default_rng(seed)
    dates = pd.date_range(start, end, freq="MS")
    n = len(dates)
    t = np.arange(n)
    month = pd.DatetimeIndex(dates).month.to_numpy()

    # ── Corn price (target): trend + seasonality + shocks ──────────
    # Real corn ranged ~$2–$8.50/bu from 2000–2024
    trend       = 2.0 + t * 0.005                                   # slow upward trend
    season_p    = 0.3 * np.sin(2 * np.pi * (month - 3) / 12)       # seasonal dip at harvest
    shocks      = rng.normal(0, 0.15, n)
    shocks[120] += 2.2   # 2010–2012 drought spike
    shocks[60]  += 0.8   # 2005 ethanol boom
    shocks[230] -= 0.6   # 2019 trade war dip
    price       = trend + season_p + np.cumsum(shocks * 0.3) + 2.5
    price       = np.clip(price, 1.8, 9.0).astype(float)

    # ── USDA: Yield (bu/acre) ───────────────────────────────────────
    # National avg ~150–185 bu/acre; varies by weather, genetics
    base_yield  = 140 + t * 0.18
    yield_shock = rng.normal(0, 8, n)
    yield_shock[np.where((month == 8) | (month == 9))[0]] += \
        rng.choice([-15, -10, 0, 5], size=np.sum((month == 8) | (month == 9)))
    corn_yield  = base_yield + yield_shock
    corn_yield  = np.clip(corn_yield, 100, 200)

    # ── USDA: Planted acreage (million acres) ───────────────────────
    base_acres  = 75 + rng.normal(0, 3, n)
    # Corn/soy rotation: acreage varies with relative prices
    acres_adj   = base_acres + 0.5 * (price - price.mean()) / price.std()
    planted_acres = np.clip(acres_adj, 60, 95)

    # ── USDA: Ending stocks-to-use ratio (%) ───────────────────────
    # Low S/U → higher prices. Inversely correlated with price.
    stu = 15 - 1.5 * (price - price.mean()) / price.std() + rng.normal(0, 2, n)
    stu = np.clip(stu, 3, 35)


    # ── NOAA: Growing season temp anomaly (°C from avg) ────────────
    temp_anomaly = rng.normal(0, 0.8, n)
    temp_anomaly += 0.015 * t / 12    # warming trend

    # ── NOAA: Precipitation anomaly (% of normal) ──────────────────
    precip_anomaly = rng.normal(100, 15, n)
    # 2012 drought
    precip_anomaly[140:155] -= 35


    # ── EIA: WTI Crude Oil ($/bbl) ──────────────────────────────────
    oil_trend  = 30 + t * 0.18
    oil_season = 5 * np.sin(2 * np.pi * month / 12)
    oil_shock  = np.cumsum(rng.normal(0, 2.5, n))
    crude_oil  = np.clip(oil_trend + oil_season + oil_shock * 0.2, 20, 130)

    # ── EIA: Ethanol production (million gallons/month) ─────────────
    ethanol = 200 + t * 4.5 + rng.normal(0, 20, n)
    ethanol = np.clip(ethanol, 180, 1600)


    # ── FRED: USD Index (DXY-equivalent, broad) ─────────────────────
    usd_index = 110 + np.cumsum(rng.normal(0, 0.5, n)) * 0.3
    usd_index = np.clip(usd_index, 85, 135)

    # ── FRED: Federal Funds Rate (%) ────────────────────────────────
    fed_funds = np.clip(5 - t * 0.015 + rng.normal(0, 0.3, n), 0.05, 8.0)

    # ── FRED: CPI YoY (%) ───────────────────────────────────────────
    cpi = 2.0 + rng.normal(0, 0.5, n) + np.clip((t - 240) * 0.04, 0, 7)


    # ── CFTC: Managed Money Net Position (contracts, thousands) ─────
    mm_net = 150 + 80 * np.sin(2 * np.pi * t / 36) + rng.normal(0, 30, n)
    mm_net += 50 * (price - price.mean()) / price.std()   # momentum effect

    # ── Soybean/Corn ratio (planting decision signal) ───────────────
    soy_price   = price * (2.5 + rng.normal(0, 0.2, n))
    soy_corn_ratio = soy_price / price

    # ── Natural gas (fertilizer cost proxy) ($/MMBtu) ───────────────
    nat_gas = 3.5 + np.cumsum(rng.normal(0, 0.1, n)) * 0.15
    nat_gas = np.clip(nat_gas, 1.5, 12.0)

    # ── Crop progress: % harvested by month ─────────────────────────
    harvest_pct = np.where(month == 10, 50,
                  np.where(month == 11, 90,
                  np.where(month == 12, 98, 5))).astype(float)
    harvest_pct += rng.normal(0, 3, n)
    harvest_pct = np.clip(harvest_pct, 0, 100)

    df = pd.DataFrame({
        "date":             dates,
        "corn_price":       price,          # TARGET ($/bushel)
        # USDA
        "yield_bu_acre":    corn_yield,
        "planted_acres_m":  planted_acres,
        "stocks_to_use":    stu,
        # NOAA
        "temp_anomaly_c":   temp_anomaly,
        "precip_pct_normal":precip_anomaly,
        # EIA
        "crude_oil_usd":    crude_oil,
        "ethanol_prod_mgal":ethanol,
        "nat_gas_usd":      nat_gas,
        # FRED
        "usd_index":        usd_index,
        "fed_funds_rate":   fed_funds,
        "cpi_yoy":          cpi,
        # CFTC
        "mm_net_pos_k":     mm_net,
        # Derived
        "soy_corn_ratio":   soy_corn_ratio,
        "harvest_pct":      harvest_pct,
        "month":            month,
    }).set_index("date")

    return df

# ─────────────────────────────────────────────────────────────
# 2.  REAL DATA COLLECTION
# ─────────────────────────────────────────────────────────────

def collect_real_datasets(start="2000-01-01", end="2025-12-31", seed=42):
    """
    Generate a realistic monthly dataset of corn price drivers.
    Statistical properties calibrated to historical USDA/NOAA/EIA data.
    """
    rng = np.random.default_rng(seed)
    dates = pd.date_range(start, end, freq="MS")
    n = len(dates)
    t = np.arange(n)
    month = pd.DatetimeIndex(dates).month.to_numpy()

    # ── Corn price (target): trend + seasonality + shocks ──────────
    # Real corn ranged ~$2–$8.50/bu from 2000–2024
    import yfinance as yf

    # Load the commodity price data
    price_df = yf.download('ZC=F', start=start, end=end, interval='1mo')
    price_df.head()

    price = price_df["Close"]
    #price = np.clip(price, 1.8, 9.0).astype(float)

    # ── USDA
    usda_df = fetch_usda_nass()
    usda_df.head()

    # ── USDA: Yield (bu/acre) ───────────────────────────────────────
    # National avg ~150–185 bu/acre; varies by weather, genetics
    corn_yield  = usda_df["yield_bu_acre"]

    # ── USDA: Planted acreage (million acres) ───────────────────────


    # ── USDA: Ending stocks-to-use ratio (%) ───────────────────────


    # ── NOAA
    #noaa_df = fetch_noaa_climate()
    #noaa_df.head()

    # ── NOAA: Growing season temp anomaly (°C from avg) ────────────
    #temp_anomaly = noaa_df[""]

    # ── NOAA: Precipitation anomaly (% of normal) ──────────────────
    #precip_anomaly = noaa_df[""]


    # ── EIA
    eia_df = fetch_eia()
    eia_df.head()

    # ── EIA: WTI Crude Oil ($/bbl) ──────────────────────────────────
    crude_oil = eia_df["PET.RWTC.M"]

    # ── Natural gas (fertilizer cost proxy) ($/MMBtu) ───────────────
    #nat_gas = fetch_eia(series_id = """)

    # ── EIA: Ethanol production (million gallons/month) ─────────────
    #ethanol_df = fetch_eia(series_id = "STEO.CORNPROD_US.M")
    #ethanol = ethanol_df["STEO.CORNPROD_US.M"]


    # ── FRED
    fred_df = fetch_fred()
    fred_df.head()

    # ── FRED: USD Index (DXY-equivalent, broad) ─────────────────────
    usd_index = fred_df["DTWEXBGS"]

    # ── FRED: Federal Funds Rate (%) ────────────────────────────────
    fed_funds = fred_df["FEDFUNDS"]

    # ── FRED: CPI YoY (%) ───────────────────────────────────────────
    cpi = fed_funds = fred_df["CPIAUCSL"]


    # ── CFTC
    #cftc_df = fetch_cftc_cot()
    #cftc_df.head()

    # ── CFTC: Managed Money Net Position (contracts, thousands) ─────
    #mn_net = cftc_df[""]


    # ── Soybean/Corn ratio (planting decision signal) ───────────────
    soy_price_df = yf.download('ZS=F', start=start, end=end, interval='1mo')
    soy_price_df.head()

    soy_price = soy_price_df["Close"]
    soy_corn_ratio = soy_price / price


    # ── Crop progress: % harvested by month ─────────────────────────
    harvest_pct = np.where(month == 10, 50,
                  np.where(month == 11, 90,
                  np.where(month == 12, 98, 5))).astype(float)
    harvest_pct += rng.normal(0, 3, n)
    harvest_pct = np.clip(harvest_pct, 0, 100)

    df = pd.DataFrame({
        "date":             dates,
        "corn_price":       price,          # TARGET ($/bushel)
        # USDA
        "yield_bu_acre":    corn_yield,
        #"planted_acres_m":  planted_acres,
        #"stocks_to_use":    stu,
        # NOAA
        #"temp_anomaly_c":   temp_anomaly,
        #"precip_pct_normal":precip_anomaly,
        # EIA
        "crude_oil_usd":    crude_oil,
        #"ethanol_prod_mgal":ethanol,
        #"nat_gas_usd":      nat_gas,
        # FRED
        "usd_index":        usd_index,
        "fed_funds_rate":   fed_funds,
        "cpi_yoy":          cpi,
        # CFTC
        #"mm_net_pos_k":     mm_net,
        # Derived
        "soy_corn_ratio":   soy_corn_ratio,
        "harvest_pct":      harvest_pct,
        "month":            month,
    }).set_index("date")

    return df

In [23]:
# ─────────────────────────────────────────────────────────────
# 3.  FEATURE ENGINEERING
# ─────────────────────────────────────────────────────────────

def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Build lag features, rolling statistics, and derived signals.
    """
    df = df.copy()

    base_features = ["yield_bu_acre", "crude_oil_usd", "usd_index", "fed_funds_rate", "cpi_yoy", "soy_corn_ratio", "harvest_pct"]#, "planted_acres_m", "stocks_to_use", "temp_anomaly_c", "precip_pct_normal", "ethanol_prod_mgal", "nat_gas_usd", "mm_net_pos_k"]

    # Lag features: t-1, t-2, t-3, t-6, t-12
    for feat in base_features:
        for lag in [1, 2, 3, 6, 12]:
            df[f"{feat}_lag{lag}"] = df[feat].shift(lag)

    # Rolling statistics on price (momentum)
    for window in [3, 6, 12]:
        df[f"price_ma{window}"]   = df["corn_price"].shift(1).rolling(window).mean()
        df[f"price_std{window}"]  = df["corn_price"].shift(1).rolling(window).std()
        df[f"price_mom{window}"]  = df["corn_price"].shift(1).pct_change(window)

    # Rolling stats on key drivers
    for feat in ["crude_oil_usd", "mm_net_pos_k", "stocks_to_use"]:
        df[f"{feat}_ma3"]  = df[feat].shift(1).rolling(3).mean()
        df[f"{feat}_diff1"] = df[feat].diff(1)

    # Calendar: month dummies capture seasonality
    for m in range(1, 13):
        df[f"month_{m}"] = (df["month"] == m).astype(int)

    # Interaction: oil × USD (dollar-denominated commodity effect)
    df["oil_x_usd"] = df["crude_oil_usd_lag1"] * df["usd_index_lag1"]

    # Ratio: stocks-to-use momentum
    df["stu_chg3"] = df["stocks_to_use"].diff(3)

    # Drop the raw month column (encoded as dummies above)
    df = df.drop(columns=["month"])

    return df

In [5]:
# ─────────────────────────────────────────────────────────────
# 4.  WALK-FORWARD CROSS-VALIDATION
# ─────────────────────────────────────────────────────────────

def walk_forward_cv(model, X: pd.DataFrame, y: pd.Series,
                    min_train_months=60, step=6):
    """
    Time-series walk-forward validation.
    Train on expanding window, predict next `step` months.
    Returns predictions DataFrame aligned to y's index.
    """
    preds = pd.Series(index=y.index, dtype=float)
    indices = np.arange(len(y))

    for split in range(min_train_months, len(y) - step + 1, step):
        train_idx = indices[:split]
        test_idx  = indices[split:split + step]

        X_tr, y_tr = X.iloc[train_idx], y.iloc[train_idx]
        X_te        = X.iloc[test_idx]

        model.fit(X_tr, y_tr)
        preds.iloc[test_idx] = model.predict(X_te)

    return preds

In [6]:
# ─────────────────────────────────────────────────────────────
# 5.  MODEL TRAINING & EVALUATION
# ─────────────────────────────────────────────────────────────

def train_and_evaluate(df_feat: pd.DataFrame):
    """
    Train three models (Ridge, Random Forest, Gradient Boosting),
    run walk-forward CV, compare metrics, pick winner.
    """
    target = "corn_price"
    feature_cols = [c for c in df_feat.columns if c != target]

    df_model = df_feat[[target] + feature_cols].dropna()
    y = df_model[target]
    X = df_model[feature_cols]

    print(f"\n{'='*60}")
    print(f"  Dataset: {len(X)} months  |  Features: {len(feature_cols)}")
    print(f"  Date range: {X.index[0].date()} → {X.index[-1].date()}")
    print(f"{'='*60}\n")

    models = {
        "Ridge Regression": Pipeline([
            ("scaler", StandardScaler()),
            ("model",  Ridge(alpha=10.0))
        ]),
        "Random Forest": RandomForestRegressor(
            n_estimators=200, max_depth=8, min_samples_leaf=5,
            random_state=42, n_jobs=-1
        ),
        "Gradient Boosting": GradientBoostingRegressor(
            n_estimators=300, max_depth=4, learning_rate=0.05,
            subsample=0.8, min_samples_leaf=5, random_state=42
        ),
    }

    results = {}
    all_preds = {}

    for name, model in models.items():
        print(f"  Walk-forward CV: {name}...")
        preds = walk_forward_cv(model, X, y, min_train_months=60, step=6)
        valid = preds.dropna()
        y_valid = y.loc[valid.index]

        mae  = mean_absolute_error(y_valid, valid)
        rmse = np.sqrt(mean_squared_error(y_valid, valid))
        r2   = r2_score(y_valid, valid)
        mape = np.mean(np.abs((y_valid - valid) / y_valid)) * 100

        results[name] = {"MAE": mae, "RMSE": rmse, "R²": r2, "MAPE%": mape}
        all_preds[name] = preds

        print(f"     MAE={mae:.3f}  RMSE={rmse:.3f}  R²={r2:.3f}  MAPE={mape:.1f}%")

    # ── Train best model on full data for feature importance ────────
    best_name = max(results, key=lambda k: results[k]["R²"])
    print(f"\n  Best model: {best_name} (R²={results[best_name]['R²']:.3f})\n")

    best_model = models[best_name]
    best_model.fit(X, y)

    return best_model, best_name, results, all_preds, X, y

In [7]:
# ─────────────────────────────────────────────────────────────
# 6.  FEATURE IMPORTANCE
# ─────────────────────────────────────────────────────────────

def get_feature_importance(model, model_name, X, y, top_n=20):
    """Extract feature importances (works for RF/GB natively, Ridge via permutation)."""
    if hasattr(model, "feature_importances_"):
        imp = pd.Series(model.feature_importances_, index=X.columns)
    elif hasattr(model, "named_steps"):
        # Pipeline (Ridge)
        result = permutation_importance(model, X, y, n_repeats=10,
                                        random_state=42, n_jobs=-1)
        imp = pd.Series(result.importances_mean, index=X.columns)
    else:
        result = permutation_importance(model, X, y, n_repeats=10,
                                        random_state=42, n_jobs=-1)
        imp = pd.Series(result.importances_mean, index=X.columns)

    return imp.nlargest(top_n)

In [8]:
# ─────────────────────────────────────────────────────────────
# 7.  PLOTTING
# ─────────────────────────────────────────────────────────────

def plot_results(df_feat, all_preds, results, feature_imp, best_name, output_path):
    """Generate a comprehensive 4-panel results figure."""
    y = df_feat["corn_price"]

    fig = plt.figure(figsize=(18, 14), facecolor="#0f1117")
    gs  = gridspec.GridSpec(3, 2, figure=fig, hspace=0.45, wspace=0.35)

    GOLD  = "#FFD700"
    GREEN = "#00C896"
    RED   = "#FF4B4B"
    BLUE  = "#4B9FFF"
    BG    = "#0f1117"
    PANEL = "#1a1d27"
    TEXT  = "#E8E8E8"

    plt.rcParams.update({
        "axes.facecolor": PANEL, "figure.facecolor": BG,
        "axes.edgecolor": "#333", "axes.labelcolor": TEXT,
        "xtick.color": TEXT, "ytick.color": TEXT,
        "text.color": TEXT, "grid.color": "#2a2d37",
        "grid.alpha": 0.5,
    })

    # ── Panel 1: Actual vs. Predicted (best model) ──────────────────
    ax1 = fig.add_subplot(gs[0, :])
    best_preds = all_preds[best_name].dropna()
    ax1.plot(y.index, y.values, color=GOLD, linewidth=1.5, label="Actual Price", zorder=3)
    ax1.plot(best_preds.index, best_preds.values, color=GREEN,
             linewidth=1.5, linestyle="--", label=f"Predicted ({best_name})", zorder=3)
    ax1.fill_between(best_preds.index, y.loc[best_preds.index], best_preds.values,
                     alpha=0.15, color=RED)
    ax1.set_title("Corn Futures Price — Actual vs. Walk-Forward CV Predictions",
                  fontsize=13, fontweight="bold", color=GOLD, pad=10)
    ax1.set_ylabel("Price ($/bushel)", fontsize=10)
    ax1.legend(loc="upper left", framealpha=0.3, fontsize=9)
    ax1.grid(True, axis="y")
    # Shade drought period
    ax1.axvspan(pd.Timestamp("2011-06-01"), pd.Timestamp("2013-01-01"),
                alpha=0.1, color=RED, label="2012 Drought")

    # ── Panel 2: Model Comparison Bar ───────────────────────────────
    ax2 = fig.add_subplot(gs[1, 0])
    metrics_df = pd.DataFrame(results).T
    x = np.arange(len(metrics_df))
    colors = [GREEN if n == best_name else BLUE for n in metrics_df.index]
    bars = ax2.bar(x, metrics_df["R²"], color=colors, width=0.5, alpha=0.85)
    ax2.set_xticks(x)
    ax2.set_xticklabels([n.replace(" ", "\n") for n in metrics_df.index], fontsize=8)
    ax2.set_ylim(0, 1.0)
    ax2.set_title("Model R² (Walk-Forward CV)", fontsize=11, fontweight="bold")
    ax2.set_ylabel("R²")
    ax2.grid(True, axis="y")
    for bar, r2 in zip(bars, metrics_df["R²"]):
        ax2.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01,
                 f"{r2:.3f}", ha="center", va="bottom", fontsize=9, color=TEXT)

    # Metrics table
    tbl_data = [[f"{v['MAE']:.3f}", f"{v['RMSE']:.3f}", f"{v['MAPE%']:.1f}%"]
                for v in results.values()]
    ax2.table(cellText=tbl_data,
              rowLabels=[n.replace(" ", "\n") for n in results.keys()],
              colLabels=["MAE", "RMSE", "MAPE"],
              cellLoc="center", loc="bottom", bbox=[0, -0.55, 1, 0.45])

    # ── Panel 3: Feature Importance ─────────────────────────────────
    ax3 = fig.add_subplot(gs[1, 1])
    fi = feature_imp.sort_values()
    colors_fi = [GOLD if i >= len(fi) - 5 else BLUE for i in range(len(fi))]
    ax3.barh(fi.index, fi.values, color=colors_fi, alpha=0.85)
    ax3.set_title(f"Top {len(fi)} Feature Importances\n({best_name})",
                  fontsize=11, fontweight="bold")
    ax3.set_xlabel("Importance Score")
    ax3.tick_params(axis="y", labelsize=7)
    ax3.grid(True, axis="x")

    # ── Panel 4: Residual Analysis ───────────────────────────────────
    ax4 = fig.add_subplot(gs[2, 0])
    residuals = y.loc[best_preds.index] - best_preds
    ax4.scatter(best_preds, residuals, alpha=0.4, s=15, color=BLUE)
    ax4.axhline(0, color=GOLD, linewidth=1.5, linestyle="--")
    ax4.set_title("Residuals vs. Predicted", fontsize=11, fontweight="bold")
    ax4.set_xlabel("Predicted Price ($/bu)")
    ax4.set_ylabel("Residual ($/bu)")
    ax4.grid(True)

    # ── Panel 5: Residual Distribution ──────────────────────────────
    ax5 = fig.add_subplot(gs[2, 1])
    ax5.hist(residuals, bins=40, color=BLUE, alpha=0.75, edgecolor="#333")
    ax5.axvline(residuals.mean(), color=GOLD, linewidth=2, label=f"Mean: {residuals.mean():.3f}")
    ax5.axvline(residuals.median(), color=GREEN, linewidth=2,
                linestyle="--", label=f"Median: {residuals.median():.3f}")
    ax5.set_title("Residual Distribution", fontsize=11, fontweight="bold")
    ax5.set_xlabel("Residual ($/bu)")
    ax5.set_ylabel("Frequency")
    ax5.legend(fontsize=9)
    ax5.grid(True, axis="y")

    fig.suptitle("🌽  CORN COMMODITY PRICE PREDICTION  |  ML Pipeline Results",
                 fontsize=16, fontweight="bold", color=GOLD, y=0.98)

    plt.savefig(output_path, dpi=150, bbox_inches="tight", facecolor=BG)
    plt.close()
    print(f"  Plot saved → {output_path}")

In [9]:
# ─────────────────────────────────────────────────────────────
# 8.  FORECASTING
# ─────────────────────────────────────────────────────────────

def forecast_next_n_months(model, df_feat, n_months=6):
    """
    Iteratively forecast the next n months using the trained model.
    Each step uses the previous prediction as input for lag features.
    """
    df_copy  = df_feat.copy()
    feature_cols = [c for c in df_copy.columns if c != "corn_price"]
    forecasts = []

    for i in range(n_months):
        last_row = df_copy[feature_cols].iloc[[-1]].copy()
        pred = model.predict(last_row.ffill())[0]

        next_date = df_copy.index[-1] + pd.DateOffset(months=1)
        new_row = df_copy.iloc[-1].copy()
        new_row.name = next_date
        new_row["corn_price"] = pred

        # Shift lag features forward one step
        for feat in [c for c in df_copy.columns
                     if not c.startswith("month_") and c != "corn_price"
                     and not any(x in c for x in ["_lag", "_ma", "_std", "_mom"])]:
            if f"{feat}_lag1" in df_copy.columns:
                new_row[f"{feat}_lag1"] = df_copy[feat].iloc[-1]
            if f"{feat}_lag2" in df_copy.columns:
                new_row[f"{feat}_lag2"] = df_copy[f"{feat}_lag1"].iloc[-1]
            if f"{feat}_lag3" in df_copy.columns:
                new_row[f"{feat}_lag3"] = df_copy[f"{feat}_lag2"].iloc[-1]

        df_copy = pd.concat([df_copy, pd.DataFrame([new_row])])
        forecasts.append({"date": next_date, "forecast_price": round(pred, 4)})

    return pd.DataFrame(forecasts).set_index("date")

In [26]:
# ─────────────────────────────────────────────────────────────
# 9.  MAIN
# ─────────────────────────────────────────────────────────────

def main():
    print("\n" + "="*60)
    print("  🌽  CORN PRICE PREDICTION ML PIPELINE")
    print("="*60)

    # ── Step 1: Load data ────────────────────────────────────────────
    print("\n[1/5]  Generating dataset...")
    df_raw = collect_real_datasets()
    print(f"       {len(df_raw)} monthly observations, {df_raw.shape[1]} raw features")

    # ── Step 2: Feature engineering ─────────────────────────────────
    print("\n[2/5]  Engineering features...")
    df_feat = engineer_features(df_raw)
    print(f"       {df_feat.shape[1]} total features after lags/rolling stats")

    # ── Step 3: Train & evaluate ─────────────────────────────────────
    print("\n[3/5]  Training models (walk-forward CV)...")
    best_model, best_name, results, all_preds, X, y = train_and_evaluate(df_feat)

    # ── Step 4: Feature importance ───────────────────────────────────
    print(f"\n[4/5]  Computing feature importances ({best_name})...")
    feature_imp = get_feature_importance(best_model, best_name, X, y, top_n=20)
    print("\n  Top 10 most predictive features:")
    for feat, score in feature_imp.head(10).items():
        print(f"     {feat:<40} {score:.4f}")

    # ── Step 5: Plot & forecast ──────────────────────────────────────
    print("\n[5/5]  Generating plots and 6-month forecast...")
    plot_results(df_feat.dropna(), all_preds, results, feature_imp, best_name,
                 "/mnt/user-data/outputs/corn_price_model_results.png")

    forecast_df = forecast_next_n_months(best_model, df_feat.dropna(), n_months=6)
    print("\n  📈  6-Month Corn Price Forecast ($/bushel):")
    print("  " + "-"*35)
    for date, row in forecast_df.iterrows():
        print(f"     {date.strftime('%b %Y')}:  ${row['forecast_price']:.2f}/bu")
    print("  " + "-"*35)

    # ── Save results to CSV ──────────────────────────────────────────
    forecast_df.to_csv("/mnt/user-data/outputs/corn_price_forecast.csv")

    # Save model metrics
    pd.DataFrame(results).T.round(4).to_csv(
        "/mnt/user-data/outputs/corn_model_metrics.csv")

    print("\n  ✅  Pipeline complete. Outputs saved to /mnt/user-data/outputs/")
    print("="*60 + "\n")


if __name__ == "__main__":
    main()


  🌽  CORN PRICE PREDICTION ML PIPELINE

[1/5]  Generating dataset...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


ValueError: array length 312 does not match index length 680